# Assignment 4 - Using NLP to play the stock market

In this assignment, we'll use everything we've learned to analyze corporate news and pick stocks. Be aware that in this assignment, we're trying to beat the benchmark of random chance (aka better than 50%).

This assignment will involve building three models:

**1. An RNN based on word inputs**

**2. A CNN based on character inputs**

**3. A neural net architecture that merges the previous two models**

You will apply these models to predicting whether a stock return will be positive or negative in the same day of a news publication.

## Your X - Reuters news data

Reuters is a news outlet that reports on corporations, among many other things. Stored in the `news_reuters.csv` file is news data listed in columns. The corresponding columns are the `ticker`, `name of company`, `date of publication`, `headline`, `first sentence`, and `news category`.

In this assignment it is up to you to decide how to clean this dataset. For instance, many of the first sentences contain a location name showing where the reporting is done. This is largely irrevant information and will probably just make your data noisier. You can also choose to subset on a certain news category, which might enhance your model performance and also limit the size of your data.

## Your Y - Stock information from Yahoo! Finance

Trading data from Yahoo! Finance was collected and then normalized using the [S&P 500](https://en.wikipedia.org/wiki/S%26P_500_Index). This is stored in the `stockReturns.json` file. 

In our dataset, the ticker for the S&P is `^GSPC`. Each ticker is compared the S&P and then judged on whether it is outperforming (positive value) or under-performing (negative value) the S&P. Each value is reported on a daily interval from 2004 to now.

Below is a diagram of the data in the json file. Note there are three types of data: short: 1 day return, mid: 7 day return, long 28 day return.

```
          term (short/mid/long)
         /         |         \
   ticker A   ticker B   ticker C
      /   \      /   \      /   \
  date1 date2 date1 date2 date1 date2
```

You will need to pick a length of time to focus on (day, week, month). You are welcome to train models on each dataset as well.  

Transform the return data such that the outcome will be binary:

```
label[y < 0] = 0
label[y >= 0] = 1
```

Finally, this data needs needs to be joined on the date and ticker - For each date of news publication, we want to join the corresponding corporation's news on its return information. We make the assumption that the day's return will reflect the sentiment of the news, regardless of timing.


# Your models - RNN, CNN, and RNN+CNN

For your RNN model, it needs to be based on word inputs, embedding the word inputs, encoding them with an RNN layer, and finally a decoding step (such as softmax or some other choice).

Your CNN model will be based on characters. For reference on how to do this, look at the CNN class demonstration in the course repository.

Finally you will combine the architecture for both of these models, either [merging](https://github.com/ShadyF/cnn-rnn-classifier) using the [Functional API](https://keras.io/getting-started/functional-api-guide/) or [stacking](http://www.aclweb.org/anthology/S17-2134). See the links for reference.

For each of these models, you will need to:
1. Create a train and test set, retaining the same test set for every model
2. Show the architecture for each model, printing it in your python notebook
2. Report the peformance according to some metric
3. Compare the performance of all of these models in a table (precision and recall)
4. Look at your labeling and print out the underlying data compared to the labels - for each model print out 2-3 examples of a good classification and a bad classification. Make an assertion why your model does well or poorly on those outputs.
5. For each model, calculate the return from the three most probable positive stock returns. Compare it to the actual return. Print this information in a table.

### Good luck!

## Load and Clean Raw Data

In [1]:
# Utility libraries
import os
import numpy as np
import pandas as pd

# Prepocessing libraries
from sklearn.model_selection import train_test_split
import gensim


In [8]:
dataPath = '../data'
reutersFile = 'news_reuters.csv'
stockFile = 'stockReturns.json'

rawX = pd.read_csv(os.path.join(dataPath, reutersFile), header=None, 
                   names=['ticker', 'company', 'pub_date', 'headline', 'first_sent', 'category'])
rawY = pd.read_json(os.path.join(dataPath, stockFile))
# rawY = json.load(os.path.join(dataPath, stockFile))

In [10]:
rawX.loc[rawX.pub_date == 20110708]

,ticker,company,pub_date,headline,first_sent,category
1,AA,Alcoa Corporation,20110708,Global markets weekahead: Lacking conviction,LONDON Investors are unlikely to gain strong c...,normal
2,AA,Alcoa Corporation,20110708,Jobs halt Wall Street rally investors eye ear...,NEW YORK Stocks fell on Friday as a weak jobs ...,topStory
3,AA,Alcoa Corporation,20110708,REFILE-TABLE-Australia's top carbon polluters,CANBERRA July 8 Following is a list of Austr...,normal
4,AA,Alcoa Corporation,20110708,US STOCKS-Jobs data hits stocks but earnings ...,* Google slumps on downgrade one of Nasdaq's...,normal
5,AA,Alcoa Corporation,20110708,US STOCKS-Jobs halt Wall St rally investors e...,* Dow off 0.5 pct S&P down 0.7 pct Nasdaq o...,normal
6,AA,Alcoa Corporation,20110708,Wall St Week Ahead: Recipe for a rally? Beat l...,NEW YORK July 8 Wall Street heads into earni...,normal
7,AA,Alcoa Corporation,20110708,Wall St Week Ahead: Recipe for a rally? Beat l...,NEW YORK Wall Street heads into earnings seaso...,normal
1106,AAPL,Apple Inc,20110708,Analysis: Young startups demand steeper prices...,SAN FRANCISCO A year ago Mike Maples's invest...,topStory
1107,AAPL,Apple Inc,20110708,CORRECTED-UPDATE 1-TSMC UMC post lower June s...,(Corrects headline lead paragraph to show TS...,normal
1108,AAPL,Apple Inc,20110708,PRESS DIGEST - China,BEIJING/SHANGHAI July 8 Chinese newspapers a...,normal


In [53]:
def reformat_y_data(data, tickerType='mid'):
    tmp = data[tickerType].apply(pd.Series)
    tmp = tmp.stack().rename('price', inplace=True).reset_index()
    tmp['y'] = np.where(tmp1['price'] >= 0, 1, 0)
    tmp.rename(columns={'level_0': 'ticker', 'level_1': 'pub_date'}, inplace=True)
    return tmp

In [57]:
cleanY = reformat_y_data(rawY, 'mid')

In [62]:
def clean_and_merge_data(X, Y):
    """Filter X to only those tickers with stock data"""
    y_tickers = set(Y['ticker'])
    X = X.loc[X['ticker'].isin(y_tickers)]
    return X.merge(Y, on=['ticker', 'pub_date'], how='left')


In [63]:
clean_and_merge_data(rawX, cleanY)

,ticker,company,pub_date,headline,first_sent,category,price,y
0,AAPL,1-800 FLOWERSCOM Inc,20140414,Apple antitrust compliance off to a promising ...,"NEW YORK Apple Inc has made a ""promising start...",topStory,NaN,NaN
1,AAPL,1-800 FLOWERSCOM Inc,20140414,Apple antitrust compliance off to a promising ...,"NEW YORK April 14 Apple Inc has made a ""promi...",normal,NaN,NaN
2,AAPL,1-800 FLOWERSCOM Inc,20140414,COLUMN-How to avoid the trouble coming to the ...,(The opinions expressed here are those of the ...,normal,NaN,NaN
3,AAPL,1-800 FLOWERSCOM Inc,20140414,How to avoid the trouble coming to the tech se...,CHICAGO A resounding shot across the bow has b...,normal,NaN,NaN
4,AAPL,1-800 FLOWERSCOM Inc,20140415,Apple cannot escape U.S. states' e-book antitr...,NEW YORK Apple Inc on Tuesday lost an attempt ...,normal,NaN,NaN
5,AAPL,1-800 FLOWERSCOM Inc,20140415,Apple cannot escape U.S. states' e-book antitr...,NEW YORK April 15 Apple Inc on Tuesday lost a...,normal,NaN,NaN
6,AAPL,1-800 FLOWERSCOM Inc,20140415,Keep Steve Jobs' personality out of trial: tec...,SAN FRANCISCO Witnesses at an upcoming trial o...,topStory,NaN,NaN
7,AAPL,1-800 FLOWERSCOM Inc,20140415,Smartphone makers carriers embrace anti-theft...,NEW YORK April 15 Major U.S. wireless carrier...,normal,NaN,NaN
8,AAPL,1-800 FLOWERSCOM Inc,20140415,UPDATE 1-Keep Steve Jobs' personality out of t...,,normal,NaN,NaN
9,AAPL,1-800 FLOWERSCOM Inc,20140415,US STOCKS SNAPSHOT-Wall St gets lift from Coca...,NEW YORK April 15 U.S. stocks ended higher af...,normal,NaN,NaN


## Model 1: RNN

## Model 2: CNN

## Model 3: RNN+CNN